In [14]:
import pandas as pd 
import os

In [15]:
csv_path = os.path.join('..', 'Inputs', 'StationPoints.CSV')

In [16]:
os.path.exists(csv_path)

True

In [17]:
df = pd.read_csv(csv_path)

In [18]:
column_mapping = {
        'NAME': 'Key',
        'DESC': 'Name',
        'ZONEID': 'pAORGroup',
        'ALRMPRIOR': 'Priority'
    }
df = df.rename(columns=column_mapping)

In [19]:
df['Priority']

0     0
1     0
2     0
3     0
4     0
     ..
62    0
63    0
64    0
65    0
66    3
Name: Priority, Length: 67, dtype: int64

In [20]:


def scada_processing():
    print("Hello World from scada_processing!")




if __name__ == "__main__":
    scada_processing()

Hello World from scada_processing!


In [21]:
import pandas as pd
import os
from datetime import datetime

def scada_processing():
    print("Hello World from scada_processing!")
    
    # Procesar StationPoints.CSV
    df_station = process_station_points()
    if df_station is not None:
        generate_station_dat(df_station)
    
    # Procesar AnalogPoints.csv
    df_analog = process_analog_points()
    if df_analog is not None:
        print("\nPrimeras filas del DataFrame de puntos analógicos:")
        print(df_analog.head())
        generate_analog_dat(df_analog)
    
    # Procesar StatusPoints.csv
    df_status = process_status_points(df_station)
    if df_status is not None:
        print("\nPrimeras filas del DataFrame de puntos de estado:")
        print(df_status.head())
        # Aquí puedes añadir una función para generar el archivo .dat de status si es necesario

def process_station_points():
    csv_path = os.path.join('..', 'Inputs', 'StationPoints.CSV')
    
    if not os.path.exists(csv_path):
        print(f"Error: No se pudo encontrar el archivo en {csv_path}")
        return None
    
    print(f"Archivo encontrado: {csv_path}")
    df_station = pd.read_csv(csv_path)
    
    # Asegurarse de que tenemos la columna PKEY
    if 'PKEY' not in df_station.columns:
        print("Error: La columna PKEY no está presente en StationPoints.CSV")
        return None
    
    column_mapping = {
        'NAME': 'Key',
        'DESC': 'Name',
        'ZONEID': 'pAORGroup',
        'ALRMPRIOR': 'Priority'
    }
    df_station = df_station.rename(columns=column_mapping)
    df_station['Order'] = range(1, len(df_station) + 1)
    
    print("\nPrimeras filas del DataFrame de estaciones:")
    print(df_station.head())
    
    return df_station

def process_analog_points():
    # ... (el código existente se mantiene igual)

def process_status_points(df_station):
    csv_path = os.path.join('..', 'Inputs', 'StatusPoints.csv')
    
    if not os.path.exists(csv_path):
        print(f"Error: No se pudo encontrar el archivo en {csv_path}")
        return None
    
    print(f"Archivo encontrado: {csv_path}")
    df_status = pd.read_csv(csv_path)
    
    # Crear un diccionario de mapeo de NAME a PKEY desde df_station
    station_map = dict(zip(df_station['Key'], df_station['PKEY']))
    
    # Crear el nuevo DataFrame con las columnas especificadas
    df_status_new = pd.DataFrame()
    df_status_new['Type'] = 1
    df_status_new['Name'] = df_status['NAME']
    df_status_new['pStation'] = df_status['StationPID'].map(station_map)
    df_status_new['pStates'] = df_status['PREFSUFFID']
    df_status_new['pALARM_GROUP'] = 1
    df_status_new['pConfiguredAORGroup'] = df_status['USERTYPEID']
    df_status_new['ConfigNormalState'] = df_status['NORMSTATE']
    
    return df_status_new

def generate_station_dat(df_station):
    # ... (el código existente se mantiene igual)

def generate_analog_dat(df_analog):
    # ... (el código existente se mantiene igual)

if __name__ == "__main__":
    scada_processing()

IndentationError: expected an indented block after function definition on line 56 (1180869797.py, line 59)

In [1]:
def process_status_points(df_station):
    csv_path = os.path.join('..', 'Inputs', 'StatusPoints.csv')
    
    if not os.path.exists(csv_path):
        print(f"Error: Could not find the file at {csv_path}")
        return None
    
    print(f"File found: {csv_path}")
    df_status = pd.read_csv(csv_path)
    
    
    prefix_suffixes_path = os.path.join('..', 'Inputs', 'PrefixSuffixes.csv')
    if not os.path.exists(prefix_suffixes_path):
        print(f"Error: Could not find the file at {prefix_suffixes_path}")
        return None
    
    print(f"File found: {prefix_suffixes_path}")
    df_prefix_suffixes = pd.read_csv(prefix_suffixes_path)
    
    # Needed case-insensitive mapping from Name to PKey in PrefixSuffixes
    prefix_suffixes_map = {name.upper(): int(pkey) for name, pkey in zip(df_prefix_suffixes['Name'], df_prefix_suffixes['PKey'])}
    
    # case-insensitive mapping from Key to PKEY in df_station
    station_map = {key.upper(): pkey for key, pkey in zip(df_station['Key'], df_station['PKEY'])}
    
    df_status_new = pd.DataFrame()
    df_status_new['Type'] = 1  #
    df_status_new['Name'] = df_status['NAME']
    

    df_status_new['pStation'] = df_status['STATIONPID'].apply(lambda x: station_map.get(str(x).upper(), 0))
    
    # Logic for pStates (unchanged)
    df_status_new['pStates'] = df_status['PREFSUFFID'].apply(
        lambda x: prefix_suffixes_map.get(x.upper(), 0) + 200 if x else 0
    )
    
    df_status_new['pALARM_GROUP'] = 1
    df_status_new['pConfiguredAORGroup'] = df_status['USERTYPEID']
    df_status_new['ConfigNormalState'] = df_status['NORMSTATE']
    
    # Create the Key column
    def create_key(row, station_counter):
        xx = str(row['Type']).zfill(2)  # Use actual Type value, padded to 2 digits
        print(f"Type: {xx}")
        yyy = str(row['pStation']).zfill(3)  # Pad station number to 3 digits
        zzz = str(station_counter[row['pStation']]).zfill(3)  # Pad point counter to 3 digits
        station_counter[row['pStation']] += 1  # Increment counter for this station
        return f"{xx}{yyy}{zzz}"  # Return as string

    station_counter = {station: 1 for station in df_status_new['pStation'].unique()}  # Initialize counters
    df_status_new['Key'] = df_status_new.apply(lambda row: create_key(row, station_counter), axis=1)
    
    return df_status_new

IndentationError: unexpected indent (1571202779.py, line 3)